<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/07b_gettingCosines_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The file, following the base of 05_gettingCosines, serves as a support for the Logistic Regression file (07), calculating the cosines values.

In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 3.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097866 sha256=7d3638bbab4b9016a129776612dd61e1dced609bc4cd03463238d4256535e5dd
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext.util
from google.colab import files

In [ ]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('df_supp.csv') # change name depending on df

In [ ]:
import itertools

def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


# function to write in file
def log(w):
  with open('oov_log.txt', 'a+') as file:
    file.write(w)


# function to handle oov word exception
def trySimilarity(w1, w2, default, model):
  # calculate cosine similarity between two vectors if those exists in the model, else return default --> makes sense? ft deals with OOV
  try:
    v1 = model.get_sentence_vector(w1)
  except:
    log(w1)
    return default
  try:
    v2 = model.get_sentence_vector(w2)
  except:
    log(w2)
    return default
  return cos_sim(v1, v2)


def add_cosine_sim(df,col1,col2,ft_model):
  """
  Enriches dataframe with cosine similarity
  Input: a dataframe, two column names, and a fastText mdodel
  Output: The inputted dataframe with a new column with cosine similarities
  """

  #query the model and if there is an oov word write that word to oov_log.txt and move to the next word

  #add column with cosine sims row-wise

  df['Cosine'] = df.apply(lambda row: trySimilarity(row[col1], row[col2], 0, ft_model), axis=1) 

  return df

In [ ]:
df.head(2)

,Colex_pair,Prob,Colex1,Colex2,Colex_bool
0,"('chew', 'ring')",0.000016,chew,ring,0
1,"('wages', 'chop')",0.000016,wages,chop,0


In [ ]:
add_cosine_sim(df,'Concept1','Concept2', ft)

,Concept1,Concept2,Colex_pair,Colex_bool,Cosine
0,bait,ten,"('bait', 'ten')",0,0.050746
1,thursday,weather,"('thursday', 'weather')",0,0.224283
2,peg,be late,"('peg', 'be late')",0,0.108496
3,collar,flame,"('collar', 'flame')",0,0.182421
4,open,harmattan,"('open', 'harmattan')",0,0.082279
...,...,...,...,...,...
43306,grape,house,"('grape', 'house')",0,0.129280
43307,pen,soldier,"('pen', 'soldier')",0,0.151800
43308,anger,official,"('anger', 'official')",0,0.091551
43309,wolf-like animal,thousand,"('wolf-like animal', 'thousand')",0,0.100913


In [ ]:
df.to_csv('df_cosines.csv', index = False)

In [ ]:
files.download('df_cosines.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>